In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRdcrJ2bVyZOkBidJleqA74wYZdhcxrl1QkkBGBM4RK9GlNpfl0si3bup6Yw9UtYxBK6fo&usqp=CAU)pysmash.wordpress.com

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df = pd.read_csv('../input/cusersmarildownloadsgermancsv/german.csv', delimiter=';', encoding = "ISO-8859-2", nrows = nRowsRead)
df.dataframeName = 'german.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head().style.set_properties(**{'background-color':'bisque',
                                     'color': 'purple'})


#That's the 18th Notebook in this Dataset. No missing values.

#https://lazypredict.readthedocs.io/en/latest/

#https://pypi.org/project/lazypredict/

#Code by Srikar Dornala https://www.kaggle.com/srikardornala/standard-way-to-approach-regression-problems/notebook

In [ ]:
import plotly.graph_objects as go
corr = df.corr()
graph = go.Figure()
graph.add_trace(go.Heatmap(z=corr.values, x=corr.index.values, y=corr.columns.values))
graph.show()

#Check for imbalance in the dataset

In [ ]:
!pip install smogn

#Below I chose Creditability, since all values are 1 (phi). I changed to Duration_of_Credit_monthly that have many different values.

Value Error: redefine phi relevance function: all points are 1

In [ ]:
import smogn
concrete_smogn = smogn.smoter(
    data = df,       
    y = 'Duration_of_Credit_monthly'  
)

In [ ]:
sns.kdeplot(df['Duration_of_Credit_monthly'], label = "Original")
sns.kdeplot(concrete_smogn['Duration_of_Credit_monthly'], label = "Modified")

#The step above is not compulsory for regression. However the accuracy of classification problems increases by using SMOTE.

In [ ]:
#splitting the data
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

#Since we are using a decision tree model , we don't need to use StandardScaler

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
dtr.score(X_test, y_test)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
dtr1 = DecisionTreeRegressor()
dtr1.fit(X_train, y_train)
dtr1.score(X_test, y_test)

#As we can see there's no major improvement in the performance of the model, now to go for hyperparameter optimization

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'criterion' : [ 'mae' ,'mse', 'friedman_mse','poisson'],
    'splitter' : ['best', 'random'],
    'max_depth' : [3, 5, 7, 9, 10,12,],
    'min_samples_split' : [ 2, 3, 4, 5,7],
    'min_samples_leaf' : [ 2, 3, 4, 5,7]
}

grid_search = GridSearchCV(dtr1, grid_params, cv = 5, n_jobs = -1, verbose = 1)
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_params_)
print(grid_search.best_score_)

#An alternative approach to select a base model is to use the lazyregressor from lazypredict

https://pypi.org/project/lazypredict/

In [ ]:
!pip install lazypredict

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.19.1 which is incompatible.
sklearn-pandas 2.1.0 requires pandas>=1.1.4, but you have pandas 1.0.5 which is incompatible.
pyldavis 3.3.1 requires numpy>=1.20.0, but you have numpy 1.19.1 which is incompatible.
pyldavis 3.3.1 requires pandas>=1.2.0, but you have pandas 1.0.5 which is incompatible.
plotnine 0.8.0 requires pandas>=1.1.0, but you have pandas 1.0.5 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.1 which is incompatible.
osmnx 1.0.1 requires pandas>=1.1, but you have pandas 1.0.5 which is incompatible.
mizani 0.7.3 requires pandas>=1.1.0, but you have pandas 1.0.5 which is incompatible.
matrixprofile 1.1.10 requires protobuf==3.11.2, but you have protobuf 3.15.8 which is incompatible.
imbalanced-learn 0.8.0 requires scikit-learn>=0.24, but you have scikit-learn 0.23.1 which is incompatible.
autogluon-core 0.1.0 requires numpy==1.19.5, but you have numpy 1.19.1 which is incompatible.

In [ ]:
import lazypredict

#After installing lazypredict you need to REFRESH PAGE of your Notebook otherwise it will not work!

In [ ]:
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor(ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

#Let's see how XGB regressor (highest Adjusted R-Squared and R - Squared metric in the original code)

#I've just copied the code!

In [ ]:
from xgboost import XGBRegressor
xgbr = XGBRegressor(verbosity=0) 
xgbr.fit(X_train,y_train)
#bellow is the Adjusted R-Squared for the model
1 - (1-xgbr.score(X_test, y_test))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thank you Srikar Dornala @srikdornala for the script.' )

#I'm not Lazy though I think I ruined one more script. 

https://pypi.org/project/lazypredict/

https://lazypredict.readthedocs.io/en/latest/